# Домашнее задание

Способы выделения продуктов категории CDs&Vinyl:

1. Rule-based модель spacy (реализована). Из метаинформации понадобятся названия продуктов. Также я брала токены cd, album, disc, vinyl, record, LP и их сочетания со словами, начинающимися с заглавных букв.

2. Составить словарь сущностей (можно взять из прошлого пункта). Потом расширить его с помощью эмбеддингов, посмотрев ближайшие к ним. 

3. Можно попробовать сделать классификатор, который будет предсказывать категорию продуктов, и модель, которая будет предсказывать названия конкретных продуктов. Потом взять слова, у которых будут наибольшие веса, первая модель должна выдать что-то в духе cd, album и т.д., вторая - названия (в отличие от 1 пункта, здесь легче должны считываться неполные названия конкретных продуктов). На основе них также можно сделать rule-based модель

4. Вариант трудоемкий: сначала разметить часть данных вручную/через ruled-based модель, потом обучить на этих данных модель из allenlp, например. 

### Реализация 1 варианта:

In [2]:
import pandas as pd
import gzip
from tqdm import tqdm
import spacy
from spacy.matcher import PhraseMatcher, Matcher
import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist, bigrams
from nltk.collocations import *
from nltk.metrics.spearman import *

Создание датафреймов с отзывами и метаинформацией.

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_CDs_and_Vinyl.json.gz meta_CDs_and_Vinyl.json.gz
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_CDs_and_Vinyl_5.json.gz reviews_CDs_and_Vinyl_5.json.gz

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_CDs_and_Vinyl_5.json.gz')
meta = getDF('meta_CDs_and_Vinyl.json.gz')

Создание паттернов, извлечение наваний продуктов и биграм с ними (брала только первые 100000 отзывов, про многих нет метаинформации, поэтому всего получился корпус из 46745 отзывов).

Паттерны:

1) Названия продуктов

2) cd/album/vinyl/disc/LP/record + (опционально) ('/") Слова_с_заглавной_буквы ('/")

In [4]:
patterns = [[{'LOWER': 'cd'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'LOWER': 'album'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'LOWER': 'disc'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'LOWER': 'vinyl'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'LOWER': 'record'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}], [{'TEXT': 'LP'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}, {'IS_TITLE': 'True', 'OP': '*'}, {'TEXT': {'REGEX': '"|\''}, 'OP': '?'}]]
terms = []
rws = []
for id_p in tqdm(set(df['asin'][:100000]) & set(meta['asin'])):
    tt = meta.loc[meta['asin'] == id_p]['title'].tolist()[0]
    rw = df.loc[df['asin'] == id_p]['reviewText'].tolist()
    rws.extend(rw)
    terms.append(tt)

100%|██████████| 3028/3028 [09:56<00:00,  5.08it/s]


In [5]:
nlp = spacy.load("en_core_web_sm")
ph_matcher = PhraseMatcher(nlp.vocab)
matcher = Matcher(nlp.vocab)
ph_patterns = [nlp.make_doc(text) for text in tqdm(terms)]
ph_matcher.add('PRD', ph_patterns)
matcher.add('PRD', patterns)
docs = [nlp.make_doc(text) for text in tqdm(rws)]
matches = [matcher(doc) for doc in tqdm(docs)]
ph_matches = [ph_matcher(doc) for doc in tqdm(docs)]

100%|██████████| 46745/46745 [00:16<00:00, 2770.04it/s] 


In [6]:
ngrams = []
results = []
for match, ph_match, doc in zip(matches, ph_matches, docs):
    res = []
    for match_id, start, end in match:
        span = doc[start:end]
        res.append(span.text)
        if start > 0:
            ngrams.append((doc[start-1].text, span.text))
        if end < len(doc) - 1:
            ngrams.append((span.text, doc[end+1].text))
    for match_id, start, end in ph_match:
        span = doc[start:end]
        res.append(span.text)
        if start > 0:
            ngrams.append((doc[start-1].text, span.text))
        if end < len(doc) - 1:
            ngrams.append((span.text, doc[end+1].text))
    results.append(res)

Оценка биграмм:

In [7]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
words = []
ngs = []
for rw, r, ng in tqdm(zip(rws, results, ngrams)):
    ws = word_tokenize(rw)
    words.extend(ws)
    words.extend([w for w in r if w not in ws])
    ngs.extend(list(bigrams(ws)))
    ngs.extend([n for n in ng if n not in ngs])
w_fd = FreqDist(words)
bg_fd = FreqDist(ngs)

46745it [22:48, 34.16it/s] 


In [8]:
finder = BigramCollocationFinder(w_fd, bg_fd)
finder.apply_freq_filter(3)
nes = []
for r in results:
    nes.extend(r)
nes = set(nes)
sw = nltk.corpus.stopwords.words('english')
finder.apply_ngram_filter(lambda w1, w2: (w1 not in nes and w2 not in nes) or w1 in sw or w2 in sw or w1.isalpha() == False or w2.isalpha() == False)

In [9]:
sc_pmi = finder.score_ngrams(bigram_measures.pmi)
sc_t = finder.score_ngrams(bigram_measures.student_t)
sc_ll = finder.score_ngrams(bigram_measures.likelihood_ratio)

In [10]:
list(ranks_from_scores(sc_pmi))

[(('Reborn', 'Empowered'), 0),
 (('Semantic', 'Spaces'), 1),
 (('Django', 'Reinhardt'), 2),
 (('Faithful', 'Departed'), 3),
 (('Twenty', 'Foreplay'), 4),
 (('Greased', 'Lightnin'), 5),
 (('Unlimited', 'Capacity'), 6),
 (('Winning', 'Ugly'), 7),
 (('Graaf', 'Generator'), 8),
 (('DEEP', 'PURPLE'), 9),
 (('Siegfried', 'Idyll'), 10),
 (('Loleatta', 'Holloway'), 11),
 (('Na', 'Nog'), 12),
 (('Finyl', 'Vinyl'), 13),
 (('Alpha', 'Centauri'), 14),
 (('Flying', 'Burrito'), 15),
 (('Mask', 'Replica'), 16),
 (('Hebrew', 'Themes'), 17),
 (('Clearwater', 'Revival'), 18),
 (('Mondo', 'Generator'), 19),
 (('Restless', 'Breed'), 20),
 (('Ridiculous', 'Thoughts'), 21),
 (('Flying', 'Dutchman'), 22),
 (('Relentless', 'Beating'), 23),
 (('Ma', 'Solituda'), 24),
 (('Ma', 'Vlast'), 24),
 (('Ye', 'Faithful'), 26),
 (('Siegfried', 'Jerusalem'), 27),
 (('Flying', 'Junk'), 28),
 (('Boomtown', 'Rats'), 29),
 (('Bionic', 'Revolution'), 30),
 (('Betty', 'Boop'), 31),
 (('Orchestral', 'Favorites'), 32),
 (('Seven'

In [11]:
list(ranks_from_scores(sc_t))

[(('This', 'album'), 0),
 (('This', 'CD'), 1),
 (('first', 'album'), 2),
 (('great', 'album'), 3),
 (('The', 'album'), 4),
 (('best', 'album'), 5),
 (('debut', 'album'), 6),
 (('whole', 'album'), 7),
 (('solo', 'album'), 8),
 (('Greatest', 'Hits'), 9),
 (('good', 'album'), 10),
 (('Donna', 'Summer'), 11),
 (('live', 'album'), 12),
 (('If', 'I'), 13),
 (('second', 'album'), 14),
 (('entire', 'album'), 15),
 (('This', 'cd'), 16),
 (('studio', 'album'), 17),
 (('concept', 'album'), 18),
 (('next', 'album'), 19),
 (('album', 'cover'), 20),
 (('original', 'album'), 21),
 (('CD', 'player'), 22),
 (('double', 'album'), 23),
 (('Unforgettable', 'Fire'), 24),
 (('THIS', 'ALBUM'), 25),
 (('THIS', 'CD'), 26),
 (('album', 'ever'), 27),
 (('If', 'You'), 28),
 (('last', 'album'), 29),
 (('record', 'company'), 30),
 (('All', 'I'), 31),
 (('CD', 'set'), 32),
 (('third', 'album'), 33),
 (('great', 'CD'), 34),
 (('album', 'came'), 35),
 (('This', 'disc'), 36),
 (('classic', 'album'), 37),
 (('rock', 'al

In [12]:
list(ranks_from_scores(sc_ll))

[(('This', 'album'), 0),
 (('Greatest', 'Hits'), 1),
 (('Donna', 'Summer'), 2),
 (('This', 'CD'), 3),
 (('Unforgettable', 'Fire'), 4),
 (('debut', 'album'), 5),
 (('first', 'album'), 6),
 (('THIS', 'ALBUM'), 7),
 (('Spice', 'Girls'), 8),
 (('great', 'album'), 9),
 (('whole', 'album'), 10),
 (('King', 'Crimson'), 11),
 (('Moby', 'Grape'), 12),
 (('record', 'company'), 13),
 (('best', 'album'), 14),
 (('entire', 'album'), 15),
 (('Miles', 'Davis'), 16),
 (('Ultimate', 'Collection'), 17),
 (('CD', 'player'), 18),
 (('Hell', 'Awaits'), 19),
 (('solo', 'album'), 20),
 (('The', 'album'), 21),
 (('This', 'cd'), 22),
 (('concept', 'album'), 23),
 (('studio', 'album'), 24),
 (('THIS', 'CD'), 25),
 (('second', 'album'), 26),
 (('Black', 'Flag'), 27),
 (('Cold', 'Winter'), 28),
 (('double', 'album'), 29),
 (('live', 'album'), 30),
 (('record', 'store'), 31),
 (('original', 'LP'), 32),
 (('All', 'Eyez'), 33),
 (('X', 'Factor'), 34),
 (('original', 'vinyl'), 35),
 (('Rising', 'Force'), 36),
 (('nex

In [13]:
ne_groups = {}
for n, r in list(ranks_from_scores(sc_t)):
    for w in n:
        if w in nes:
            if w not in ne_groups:
                ne_groups[w] = [(n[0] + ' ' + n[1], r)]
            else:
                ne_groups[w].append((n[0] + ' ' + n[1], r))

In [14]:
ne_groups['album'][:5]

[('This album', 0),
 ('first album', 2),
 ('great album', 3),
 ('The album', 4),
 ('best album', 5)]

In [15]:
ne_groups['cd'][:5]

[('This cd', 16),
 ('great cd', 62),
 ('cd player', 85),
 ('good cd', 93),
 ('first cd', 160)]

In [16]:
ne_groups['disc'][:5]

[('This disc', 36),
 ('single disc', 50),
 ('second disc', 52),
 ('first disc', 79),
 ('compact disc', 99)]

In [17]:
ne_groups['vinyl'][:5]

[('original vinyl', 46),
 ('vinyl version', 114),
 ('vinyl copy', 134),
 ('vinyl LP', 159),
 ('vinyl release', 200)]

In [19]:
ne_groups['LP'][:5]

[('original LP', 42),
 ('vinyl LP', 159),
 ('double LP', 165),
 ('first LP', 217),
 ('LP set', 258)]

Комментарий к результатам: хорошо было бы добавить this + NOUN/NP в шаблоны, хотя может вылезти много лишнего

Борьба с синонимами (у меня из-за особенносетй выделения сущностей редко попадаются синонимы (в названиях могут быть синонимичные слова, но вряд ли их стоит объединять), только регистр различается, поэтому приведу в теории, как можно делать:

1) Можно попробовать воспользоваться синсетами из wordnet или его аналогов

2) Также часто можно объединять однокоренные слова/ н-граммы с общей вершиной